In [102]:
import numpy as np
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from itertools import combinations 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
# import seaborn as sns
import itertools

In [103]:
# get the train and test data from the input strings for the file locations
# ie train_data = "./data/Data1Train.csv"
def getData(train_data, test_data):
    X1_df = pd.read_csv(train_data)
    X1T_df = pd.read_csv(test_data)

    X1_train = X1_df[['x', 'y']]
    Y1_train = X1_df[['Class']]
    X1_test = X1T_df[['x', 'y']]
    Y1_test = X1T_df[['Class']]

    return X1_train, Y1_train, X1_test, Y1_test

In [104]:
# create class Node that stores its children, model used, and is it a leaf
class Node:
    def __init__(self, model):
        self.model = model
        self.isLeaf = True
        self.Left = None
        self.Right = None

    def setModel(self, model_in):
        self.model = model_in

    def setRight(self, node_in):
        self.Right = node_in
        self.isLeaf = False

    def setLeft(self, node_in):
        self.Left = node_in
        self.isLeaf = False

    def print_tree(self):
        print(self.isLeaf)
        print(self.Right.isLeaf)
        print(self.Left.isLeaf)

In [105]:
def gini_index_one(X_train, Y_train, class_label_one, model):
    class_label_one = int(class_label_one)
    w = model.coef_[0]
    b = model.intercept_[0]
    merged = X_train.join(Y_train)
    filter_points = merged[merged['Class'] == class_label_one]
    
    if w[1] == 0:
        vertical_line = -b / w[0]
        left = 0
        right = 0
        for x in filter_points['x']:
            if x < vertical_line:
                left += 1
            else:
                right += 1
        left_correct = 0
        left_wrong= 0
        right_correct= 0
        right_wrong = 0
        if left > right:
            for x, val in zip(merged['x'], merged['Class']):
                if x < vertical_line:
                    if val == class_label_one:
                        left_correct += 1
                    else:
                        left_wrong += 1
                else:
                    if val == class_label_one:
                        right_wrong += 1
                    else:
                        right_correct += 1
            left_gini = 1 - (left_correct/(left_correct+left_wrong))**2 - (left_wrong/(left_correct+left_wrong))**2
            right_gini = 1 - (right_correct/(right_correct+right_wrong))**2 - (right_wrong/(right_correct+right_wrong))**2
            return (left_correct+left_wrong)/len(merged['x']) * left_gini + (right_correct+right_wrong)/len(merged['x']) * right_gini, "below"
        else:
            for x, val in zip(merged['x'], merged['Class']):
                if x < vertical_line:
                    if val == class_label_one:
                        left_wrong += 1
                    else:
                        left_correct += 1
                else:
                    if val == class_label_one:
                        right_correct += 1
                    else:
                        right_wrong += 1
            left_gini = 1 - (left_correct/(left_correct+left_wrong))**2 - (left_wrong/(left_correct+left_wrong))**2
            right_gini = 1 - (right_correct/(right_correct+right_wrong))**2 - (right_wrong/(right_correct+right_wrong))**2
            return (left_correct+left_wrong)/len(merged['x']) * left_gini + (right_correct+right_wrong)/len(merged['x']) * right_gini, "above"
    else: 
        slope = -w[0] / w[1]
        intercept = -b / w[1]
        above = 0
        below = 0
        for x,y in zip(filter_points['x'], filter_points['y']):
            if y < slope * x + intercept:
                below += 1
            else:
                above += 1
        above_correct = 0
        above_wrong = 0
        below_correct = 0
        below_wrong = 0
        if above >= below:
            print("Gini one using class " + str(class_label_one) + " is above")
            for x, y, val in zip(merged['x'], merged['y'], merged['Class']):
                if y >= slope * x + intercept:
                    if val == class_label_one:
                        above_correct += 1
                    else:
                        above_wrong += 1
                else:
                    if val == class_label_one:
                        below_wrong += 1
                    else:
                        below_correct += 1
            above_gini = 1 - (above_correct/(above_correct+above_wrong))**2 - (above_wrong/(above_correct+above_wrong))**2
            below_gini = 1 - (below_correct/(below_correct+below_wrong))**2 - (below_wrong/(below_correct+below_wrong))**2
            return (above_correct+above_wrong)/len(merged['x']) * above_gini + (below_correct+below_wrong)/len(merged['x']) * below_gini, "above"
        else:
            above_correct = 0
            above_wrong = 0
            below_correct = 0
            below_wrong = 0
            for x, y, val in zip(merged['x'], merged['y'], merged['Class']):
                if y >= slope * x + intercept:
                    if val == class_label_one:
                        above_wrong += 1
                    else:
                        above_correct += 1
                else:
                    if val == class_label_one:
                        print(type(val))
                        print(type(class_label_one))
                        below_correct += 1
                    else:
                        below_wrong += 1
                above_gini = 1
                if (above_correct+above_wrong) != 0:
                    above_gini = 1 - (above_correct/(above_correct+above_wrong))**2 - (above_wrong/(above_correct+above_wrong))**2
                below_gini = 1
                if (below_correct+below_wrong) != 0:
                    below_gini = 1 - (below_correct/(below_correct+below_wrong))**2 - (below_wrong/(below_correct+below_wrong))**2
                return (above_correct+above_wrong)/len(merged['x']) * above_gini + (below_correct+below_wrong)/len(merged['x']) * below_gini, "below"

In [106]:
def gini_index_two(X_train, Y_train, class_label_one, class_label_two, model):
    class_label_one = int(class_label_one)
    class_label_two = int(class_label_two)
    selected = [class_label_one, class_label_two]
    w = model.coef_[0]
    b = model.intercept_[0]
    merged = X_train.join(Y_train)
    filter_points = merged[merged['Class'].isin(selected)]
    
    if w[1] == 0:
        vertical_line = -b / w[0]
        left = 0
        right = 0
        for x in filter_points['x']:
            if x < vertical_line:
                left += 1
            else:
                right += 1
        left_correct = 0
        left_wrong = 0
        right_correct = 0
        right_wrong = 0
        if left > right:
            for x, val in zip(merged['x'], merged['Class']):
                if x < vertical_line:
                    if val == class_label_one or val == class_label_two:
                        left_correct += 1
                    else:
                        left_wrong += 1
                else:
                    if val == class_label_one or val == class_label_two:
                        right_wrong += 1
                    else:
                        right_correct += 1
            left_gini = 1 - (left_correct/(left_correct+left_wrong))**2 - (left_wrong/(left_correct+left_wrong))**2
            right_gini = 1 - (right_correct/(right_correct+right_wrong))**2 - (right_wrong/(right_correct+right_wrong))**2
            return (left_correct+left_wrong)/len(merged['x']) * left_gini + (right_correct+right_wrong)/len(merged['x']) * right_gini, "below"
        else:
            for x, val in zip(merged['x'], merged['Class']):
                if x < vertical_line:
                    if val == class_label_one or val == class_label_two:
                        left_wrong += 1
                    else:
                        left_correct += 1
                else:
                    if val == class_label_one or val == class_label_two:
                        right_correct += 1
                    else:
                        right_wrong += 1
            left_gini = 1 - (left_correct/(left_correct+left_wrong))**2 - (left_wrong/(left_correct+left_wrong))**2
            right_gini = 1 - (right_correct/(right_correct+right_wrong))**2 - (right_wrong/(right_correct+right_wrong))**2
            return (left_correct+left_wrong)/len(merged['x']) * left_gini + (right_correct+right_wrong)/len(merged['x']) * right_gini, "above"
    else:
        slope = -w[0] / w[1]
        intercept = -b / w[1]
        above = 0
        below = 0
        for x,y in zip(filter_points['x'], filter_points['y']):
            if y < slope * x + intercept:
                below += 1
            else:
                above += 1
        above_correct = 0
        above_wrong = 0
        below_correct = 0
        below_wrong = 0
        if above >= below:
            for x, y, val in zip(merged['x'], merged['y'], merged['Class']):
                if y >= slope * x + intercept:
                    if val == class_label_one or val == class_label_two:
                        above_correct += 1
                    else:
                        above_wrong += 1
                else:
                    if val == class_label_one or val == class_label_two:
                        below_wrong += 1
                    else:
                        below_correct += 1
            above_gini = 1
            if (above_correct+above_wrong) != 0:
                above_gini = 1 - (above_correct/(above_correct+above_wrong))**2 - (above_wrong/(above_correct+above_wrong))**2
            below_gini = 1
            if (below_correct+below_wrong) != 0:
                below_gini = 1 - (below_correct/(below_correct+below_wrong))**2 - (below_wrong/(below_correct+below_wrong))**2
            return (above_correct+above_wrong)/len(merged['x']) * above_gini + (below_correct+below_wrong)/len(merged['x']) * below_gini, "above"
        else:
            above_correct = 0
            above_wrong = 0
            below_correct = 0
            below_wrong = 0
            for x, y, val in zip(merged['x'], merged['y'], merged['Class']):
                if y >= slope * x + intercept:
                    if val == class_label_one:
                        above_wrong += 1
                    else:
                        above_correct += 1
                else:
                    if val == class_label_one:
                        below_correct += 1
                    else:
                        below_wrong += 1
                above_gini = 1
                if (above_correct+above_wrong) != 0:
                    above_gini = 1 - (above_correct/(above_correct+above_wrong))**2 - (above_wrong/(above_correct+above_wrong))**2
                below_gini = 1
                if (below_correct+below_wrong) != 0:
                    below_gini = 1 - (below_correct/(below_correct+below_wrong))**2 - (below_wrong/(below_correct+below_wrong))**2
                return (above_correct+above_wrong)/len(merged['x']) * above_gini + (below_correct+below_wrong)/len(merged['x']) * below_gini, "below"

In [107]:
# make a boundary for the class input. determine how many points are on correct side of boundary
# place this amount over total points and square value. 1 - left - right is gini index
# train model to separate class(es) from rest of data. run gini on it to see accuracy

def separate_one(X_train, Y_train, is_not_linear, class_label_one):
    Y_bin = np.isin(Y_train, [class_label_one]).astype(int)
    Y_bin = Y_bin.ravel()
    model = SVC(kernel='linear', degree=1)
    if is_not_linear:
        model = SVC(kernel='rbf', degree = 3)
    #print(Y_bin)
    model.fit(X_train, Y_bin)
    #print(model.coef_)
    acc, direction = gini_index_one(X_train, Y_train, class_label_one, model)
    print("Gini accuracy 1 " + str(acc) + " " + direction)
    return model, acc, direction

def separate_two(X_train, Y_train, is_not_linear, class_label_one, class_label_two):
    Y_bin = np.isin(Y_train, [class_label_one, class_label_two]).astype(int)
    Y_bin = Y_bin.ravel()
    model = SVC(kernel='linear', degree=1)
    if is_not_linear:
        model = SVC(kernel='rbf', degree = 3)
    #print(Y_bin)
    model.fit(X_train, Y_bin)
    acc, direction = gini_index_two(X_train, Y_train, class_label_one, class_label_two, model)
    print("Gini accuracy 2 " + str(acc) + " " + direction)
    #print(model.coef_)
    return model, acc, direction

In [108]:
# build the tree
def build_tree(curr, classes, X_train, Y_train):
    class_pairs = list(itertools.combinations(classes, 2))
    best_model = None
    best_acc = 1.0
    best_direction = ""
    left_split = []
    right_split = []

    for val in classes:
        curr_model, curr_acc, curr_direction = separate_one(X_train, Y_train, False, val)
        if curr_acc <= best_acc:
            best_model = curr_model
            best_acc = curr_acc
            best_direction = curr_direction
            if best_direction == "below":
                left_split = np.array([val])
                right_split = classes[classes != val]
            else:
                right_split = np.array([val])
                left_split = classes[classes != val]
                
    if len(class_pairs) > 2:
        for class_label_one, class_label_two in class_pairs:
            curr_model, curr_acc, curr_direction = separate_two(X_train, Y_train, False, class_label_one, class_label_two)
            if curr_acc <= best_acc:
                best_model = curr_model
                best_acc = curr_acc
                best_direction = curr_direction
                val_arr = [class_label_one, class_label_two]
                if best_direction == "below":
                    left_split = np.array(val_arr)
                    right_split = classes[~np.isin(classes, val_arr)]
                else:
                    right_split = np.array(val_arr)
                    left_split = classes[~np.isin(classes, val_arr)]

    if len(left_split < 2) and len(right_split < 2):
        curr.setModel(best_model)
        curr.setLeft(Node(left_split[0]))
        curr.setRight(Node(right_split[0]))
        return

    merged = X_train.join(Y_train)
    filter_left = merged[merged['Class'].isin(left_split)]
    filter_right = merged[merged['Class'].isin(right_split)]
    X_train_left = filter_left[['x', 'y']]
    Y_train_left = filter_left[['Class']]
    X_train_right = filter_right[['x', 'y']]
    Y_train_right = filter_right[['Class']]
    curr.setModel(best_model)
    curr.setLeft(Node(None))
    curr.setRight(Node(None))
    build_tree(curr.Left, left_split, X_train_left, Y_train_left)
    build_tree(curr.Right, right_split, X_train_right, Y_train_right)

In [109]:
X_train, Y_train, X_test, Y_test = getData('./data/Data1Train.csv', './data/Data1Test.csv')
class_pairs = [(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]
#class_pairs = [(1, 2), (1, 3), (2, 3)]
classes = Y_train['Class'].unique()
print(type(classes))
root = Node(None)
curr = root

build_tree(curr, classes, X_train, Y_train)

<class 'numpy.ndarray'>
<class 'int'>
<class 'int'>
Gini accuracy 1 0.0 below
Gini one using class 1 is above
Gini accuracy 1 0.0 above
Gini one using class 2 is above
Gini accuracy 1 0.0 above
Gini accuracy 1 0.0 below
Gini accuracy 2 0.0 below
Gini accuracy 2 0.0 above
Gini accuracy 2 0.0 below
Gini accuracy 2 0.0 above
Gini accuracy 2 0.0 below
Gini accuracy 2 0.27776958978138055 above
